In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
in_features = 5
out_features = 2
nb_nodes = 3
W = nn.Parameter(torch.zeros(size=(in_features, out_features))) #xavier paramiter inizializator
nn.init.xavier_uniform_(W.data, gain=1.414)
input = torch.rand(nb_nodes, in_features) 
# linear transformation
h = torch.mm(input, W)
N = h.size()[0]
print(h.shape)

torch.Size([3, 2])


In [4]:
a = nn.Parameter(torch.zeros(size=(2*out_features, 1))) #xavier paramiter inizializator
nn.init.xavier_uniform_(a.data, gain=1.414)
print(a.shape)
leakyrelu = nn.LeakyReLU(0.2)  # LeakyReLU

torch.Size([4, 1])


In [5]:
a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * out_features)
a_input

tensor([[[1.5282, 0.9561, 1.5282, 0.9561],
         [1.5282, 0.9561, 1.2658, 0.4587],
         [1.5282, 0.9561, 2.0887, 1.3664]],

        [[1.2658, 0.4587, 1.5282, 0.9561],
         [1.2658, 0.4587, 1.2658, 0.4587],
         [1.2658, 0.4587, 2.0887, 1.3664]],

        [[2.0887, 1.3664, 1.5282, 0.9561],
         [2.0887, 1.3664, 1.2658, 0.4587],
         [2.0887, 1.3664, 2.0887, 1.3664]]], grad_fn=<ViewBackward0>)

In [6]:
torch.matmul(a_input, a).squeeze(2)

tensor([[ 0.2487, -0.1788,  0.9019],
        [ 0.9819,  0.5543,  1.6351],
        [-0.3755, -0.8031,  0.2777]], grad_fn=<SqueezeBackward1>)

In [7]:
e = leakyrelu(torch.matmul(a_input, a).squeeze(2))
e

tensor([[ 0.2487, -0.0358,  0.9019],
        [ 0.9819,  0.5543,  1.6351],
        [-0.0751, -0.1606,  0.2777]], grad_fn=<LeakyReluBackward0>)

In [8]:
print(a_input.shape,a.shape)
print("")
print(torch.matmul(a_input,a).shape)
print("")
print(torch.matmul(a_input,a).squeeze(2).shape)

torch.Size([3, 3, 4]) torch.Size([4, 1])

torch.Size([3, 3, 1])

torch.Size([3, 3])


In [9]:
# Masked Attention
adj = torch.randint(2, (3, 3))

zero_vec  = -9e15*torch.ones_like(e)
print(zero_vec.shape)

torch.Size([3, 3])


In [10]:
attention = torch.where(adj > 0, e, zero_vec)
print(adj,"\n",e,"\n",zero_vec)
attention

tensor([[0, 1, 1],
        [1, 0, 1],
        [0, 0, 1]]) 
 tensor([[ 0.2487, -0.0358,  0.9019],
        [ 0.9819,  0.5543,  1.6351],
        [-0.0751, -0.1606,  0.2777]], grad_fn=<LeakyReluBackward0>) 
 tensor([[-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15]])


tensor([[-9.0000e+15, -3.5769e-02,  9.0192e-01],
        [ 9.8191e-01, -9.0000e+15,  1.6351e+00],
        [-9.0000e+15, -9.0000e+15,  2.7766e-01]], grad_fn=<WhereBackward0>)

In [11]:
attention = F.softmax(attention, dim=1)
h_prime   = torch.matmul(attention, h)

In [12]:
attention

tensor([[0.0000, 0.2814, 0.7186],
        [0.3423, 0.0000, 0.6577],
        [0.0000, 0.0000, 1.0000]], grad_fn=<SoftmaxBackward0>)

In [13]:
h

tensor([[1.5282, 0.9561],
        [1.2658, 0.4587],
        [2.0887, 1.3664]], grad_fn=<MmBackward0>)

In [14]:
h_prime

tensor([[1.8571, 1.1110],
        [1.8968, 1.2260],
        [2.0887, 1.3664]], grad_fn=<MmBackward0>)

In [15]:
class GATLayer(nn.Module):
    def __init__(self, in_feat, out_feat, dropout, alpha, concat=True):
        super(GATLayer, self).__init__()
        self.dropout       = dropout        # drop prob = 0.6
        self.in_features   = in_feat 
        self.out_features  = out_feat 
        self.alpha         = alpha          # LeakyReLU with negative input slope, alpha = 0.2
        self.concat        = concat         # conacat = True for all layers except the output layer.

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        # LeakyReLU
        self.LeakyReLU = nn.LeakyReLU(self.alpha)
    
    def forward(self, input, adj):
        h = torch.mm(input, self.W)
        N = h.size()[0]
        # Attention
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e       = self.LeakyReLU(torch.matmul(a_input, self.a).squeeze(2))
        # Masked Attention
        zero_vec   = -9e15*torch.ones_like(e)
        attention  = torch.where(adj > 0, e, zero_vec)
        attention  = F.softmax(attention, dim=1)
        attention  = F.dropout(attention, self.dropout, training=self.training)
        h_prime    = torch.matmul(attention, h)
        
        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

In [17]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import matplotlib.pyplot as plt

name_data = 'Cora'
dataset=Planetoid(root=r"/home/ray/code/python/python_data_course/机器学习与深度学习导论/data",name=name_data) # root: 指定路径 name: 数据集名称
dataset.transform = T.NormalizeFeatures()

print(f"Number of Classes in {name_data}:", dataset.num_classes)
print(f"Number of Node Features in {name_data}:", dataset.num_node_features)

Number of Classes in Cora: 7
Number of Node Features in Cora: 1433


In [18]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        
        self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, dataset.num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
                
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)
    
    
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

model = GAT().to(device)
data = dataset[0].to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    
    if epoch%200 == 0:
        print(loss)
    
    loss.backward()
    optimizer.step()

tensor(1.9456, grad_fn=<NllLossBackward0>)
tensor(0.8204, grad_fn=<NllLossBackward0>)
tensor(0.6691, grad_fn=<NllLossBackward0>)
tensor(0.5005, grad_fn=<NllLossBackward0>)
tensor(0.6241, grad_fn=<NllLossBackward0>)


In [19]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8150
